In [3]:
import pickle
import time
from typing import List
import datetime as dt

import numpy as np

import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.povm import (
    Povm,
    get_x_measurement,
    get_y_measurement,
    get_z_measurement,
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_z0_1q, get_z1_1q, get_x0_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import ProjectedLinearEstimator

In [4]:
%load_ext autoreload
%autoreload 2

# True Object = State([1/np.sqrt(2), 0, 0, 1/np.sqrt(2)])

In [42]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

povm_x = get_x_measurement(c_sys)
povm_y = get_y_measurement(c_sys)
povm_z = get_z_measurement(c_sys)
povms = [povm_x, povm_y, povm_z]

In [43]:
true_object = get_z0_1q(c_sys)
num_data = [100, 1000]
iteration = 100

# Linear (on_para_eq_constraint=False)
qst = StandardQst(povms, on_para_eq_constraint=False)
estimation_results_linear = data_analysis.estimate(qtomography=qst,
                                                                         true_object=true_object,
                                                                         num_data=num_data,
                                                                         estimator=LinearEstimator(),
                                                                         iteration=iteration)

100%|██████████| 100/100 [00:01<00:00, 79.53it/s]


In [44]:
param_affine_est_linear[0].estimated_qoperation_sequence

NameError: name 'param_affine_est_linear' is not defined

In [94]:
section_1 = report._TEMPLATE_SECTION_PHYSICALITY_VIOLATION_CHECK.format(report._TAMPLATE_SUBSECTION_PYHYICALITY_VIOLATION_CHECK)
section_2 = report._TEMPLATE_SECTION_MSE

html = report._TEMPLATE_BASE.format(section_1, section_2)

In [95]:
report._convert_html2pdf(html, "hoge.pdf")

0

In [96]:
!open hoge.pdf 

## on_para_eq_constraint = False

In [17]:
violation_result = physicality_violation_check.check_physicality_violation(estimation_results_linear)
violation_result.keys()

100%|██████████| 100/100 [00:00<00:00, 6231.69it/s]


dict_keys(['sorted_eigenvalues_list', 'sum_of_eigenvalues'])

In [48]:
estimated_qoperations = [
            result.estimated_qoperation for result in estimation_results_linear
        ]
graph_list = physicality_violation_check._make_graphs_eigenvalues_state(estimated_qoperations,
                                                                        num_data=100,
                                                                        true_object=true_object)

100%|██████████| 100/100 [00:00<00:00, 5954.43it/s]


In [73]:
figs = physicality_violation_check.make_graphs_eigenvalues(estimation_results_linear, num_data=num_data, true_object=true_object)

for fig in figs:
    fig.show()

100%|██████████| 100/100 [00:00<00:00, 7167.42it/s]


In [74]:
figs = physicality_violation_check.make_graphs_sum_unphysical_eigenvalues(estimation_results_linear, num_data=num_data)

for fig in figs:
    fig.show()

100%|██████████| 100/100 [00:00<00:00, 6576.82it/s]


# True Object = State([ 1/np.sqrt(2), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6) ])

In [20]:
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

vec = np.array([1/np.sqrt(2), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6)],dtype=np.float64)
true_object = State(c_sys, vec)
num_data = [1000]  # N
iterations = 100  # Nrep

In [21]:
povm_x = get_x_measurement(c_sys)
povm_y = get_y_measurement(c_sys)
povm_z = get_z_measurement(c_sys)
povms = [povm_x, povm_y, povm_z]

In [26]:
qst = StandardQst(povms, on_para_eq_constraint=True)
estimation_results_affine = data_analysis.estimate(qtomography=qst,
                                                                         true_object=true_object,
                                                                         num_data=num_data,
                                                                         estimator=LinearEstimator(),
                                                                         iteration=iteration)

100%|██████████| 100/100 [00:01<00:00, 84.52it/s]


In [22]:
name = "case_2"
param_affine_est_linear = estimate(name, true_object, num_data, iterations, on_para_eq_constraint=True)
param_linear_est_linear = estimate(name, true_object, num_data, iterations, on_para_eq_constraint=False)

param_affine_est_linear = [p[0] for p in param_affine_est_linear]
param_linear_est_linear = [p[0] for p in param_linear_est_linear]

NameError: name 'estimate' is not defined

## on_para_eq_constraint = True

In [27]:
violation_result = physicality_violation_check.check_physicality_violation(estimation_results_affine)
violation_result.keys()

dict_keys(['violation_list'])

In [28]:
value_list = violation_result["violation_list"]
physicality_violation_check.make_prob_dist_histogram(value_list, x_range=(-1, 2), bin_size=0.01)

## on_para_eq_constraint = False

In [20]:
violation_result = physicality_violation_check.get_physicality_violation_result_for_state(param_linear_est_linear)
violation_result.keys()

100%|██████████| 1000/1000 [00:00<00:00, 7843.25it/s]


dict_keys(['sorted_eigenvalues_list', 'sum_of_eigenvalues'])

In [21]:
true_eigs = sorted([eig.real for eig in true_object.calc_eigenvalues()], reverse=True)

for i, values in enumerate(violation_result["sorted_eigenvalues_list"]):
    fig = physicality_violation_check.make_prob_dist_histogram(values, bin_size=0.0001)
    fig.update_layout(title=f"Nrep={len(values)}")
    fig.update_xaxes( title=f"Eigenvalue (i={i})")
    
    x_value = true_eigs[i]
    fig.add_shape(
                type="line",
                line_color="red",
                line_width=2,
                opacity=0.5,
                x0=x_value,
                x1=x_value,
                xref="x",
                y0=0,
                y1=1,
                yref="paper",
            )
    fig.show()

In [22]:
value_list = violation_result["sum_of_eigenvalues"]["less_than_zero"]
fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.0001, annotation_vlines=[0])

# TODO: iterationsとnum_dataを与える方法
n_rep = len(violation_result["sorted_eigenvalues_list"][0])
n = num_data[0]
title = f"N={n}, Nrep={n_rep}, Number of Unphysical estimates={len(value_list)}"
fig.update_layout(title=title)
fig.update_xaxes( title=f"Sum of unphysical eigenvalues (<0)")
fig.show()

value_list = violation_result["sum_of_eigenvalues"]["greater_than_one"]
fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.0001, annotation_vlines=[1])

n_rep = len(violation_result["sorted_eigenvalues_list"][0])
n = num_data[0]
title = f"N={n}, Nrep={n_rep}, Number of Unphysical estimates={len(value_list)}"
fig.update_layout(title=title)  # TODO
fig.update_xaxes( title=f"Sum of unphysical eigenvalues (>1)")
fig.show()

# True Object = State([1/np.sqrt(2), 0, 0, 0])

In [23]:
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

vec = np.array([1/np.sqrt(2), 0, 0, 0],dtype=np.float64)
true_object = State(c_sys, vec)
num_data = [1000]  # N
iterations = 1000  # Nrep

In [25]:
povm_x = get_x_measurement(c_sys)
povm_y = get_y_measurement(c_sys)
povm_z = get_z_measurement(c_sys)
povms = [povm_x, povm_y, povm_z]

In [26]:
name = "case_3"
param_affine_est_linear = estimate(name, true_object, num_data, iterations, on_para_eq_constraint=True)
param_linear_est_linear = estimate(name, true_object, num_data, iterations, on_para_eq_constraint=False)

param_affine_est_linear = [p[0] for p in param_affine_est_linear]
param_linear_est_linear = [p[0] for p in param_linear_est_linear]

  1%|          | 10/1000 [00:00<00:10, 97.02it/s]

time(s)=10.449997901916504


100%|██████████| 1000/1000 [00:10<00:00, 96.84it/s]

time(s)=10.330754041671753


## on_para_eq_constraint = True

In [27]:
violation_result = physicality_violation_check.get_physicality_violation_result_for_state(param_affine_est_linear)
violation_result.keys()

dict_keys(['violation_list'])

In [28]:
value_list = violation_result["violation_list"]
physicality_violation_check.make_prob_dist_histogram(value_list, x_range=(-1, 2), bin_size=0.01)

## on_para_eq_constraint = False

In [29]:
"""
(4pi / (2pi sigma^2)^3/2 ) ( lambda - lambda^true)^2  exp( - ( lambda - lambda^true)^2 / sigma^2 ), 
sigma^2 = lambda^true ( 1 - lambda^true ) / N
"""

def f(x: float, true_eigenvalue: float, num_data: int) -> float:
    sigma_2 = (true_eigenvalue * (1- true_eigenvalue))
    sigma_2 = sigma_2 / num_data
    # print(f"sigma_2: {sigma_2}")
    
    w_1 = (2 * np.pi * sigma_2) ** (3/2)  #  (2pi sigma^2)^3/2
    # print(f"w_1={w_1}")
    
    w_2 = (x - true_eigenvalue) ** 2  #  ( lambda - lambda^true)^2
    w_3 = - (1 / sigma_2) * ((x - true_eigenvalue) ** 2)  #  - ( lambda - lambda^true)^2 / sigma^2
    # print(f"w_2: {w_2}")
    
    # y = 2 * np.sqrt((2 * np.pi) / rho_2)
    y = (4 * np.pi) / w_1
    
    # print(f"y(WIP) = {y}")
    
    y = y * w_2
    
    y = y * np.exp(w_3)
    
    return y

In [30]:
print(true_object.calc_eigenvalues()[0])
f(0.51, 0.5, num_data=1000)

(0.4999999999999999+0j)


13.530450466516594

In [31]:
violation_result = physicality_violation_check.get_physicality_violation_result_for_state(param_linear_est_linear)
violation_result.keys()

100%|██████████| 1000/1000 [00:00<00:00, 8076.97it/s]


dict_keys(['sorted_eigenvalues_list', 'sum_of_eigenvalues'])

In [32]:
true_eigs = sorted([eig.real for eig in true_object.calc_eigenvalues()], reverse=True)
test_list = []
for i, values in enumerate(violation_result["sorted_eigenvalues_list"]):
    fig = physicality_violation_check.make_prob_dist_histogram(values, bin_size=0.0001)
    # True Eigenvalue
    x_value = true_eigs[i]
    fig.add_shape(
                type="line",
                line_color="red",
                line_width=2,
                opacity=0.5,
                x0=x_value,
                x1=x_value,
                xref="x",
                y0=0,
                y1=1,
                yref="paper",
            )
    
    # Line
    # x_list = sorted(fig.data[0]["x"])
    if i == 0:
        x_list = np.arange(0.5, 1.0, 0.0001)
    else:
        x_list = np.arange(0.5, 0, -0.0001)
    
    y_list = [f(x, true_eigs[i], len(values)) for x in x_list]
    line_trace = go.Scatter(x=x_list, y=y_list, line_color='rgb(0,176,246)')
    fig.add_trace(line_trace)
    test_list.append(y_list)
    
    fig.update_layout(title=f"Nrep={len(values)}")
    fig.update_xaxes( title=f"Eigenvalue (i={i})")
    fig.show()
    
    break

In [33]:
value_list = violation_result["sum_of_eigenvalues"]["less_than_zero"]
fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.0001, annotation_vlines=[0])

# TODO: iterationsとnum_dataを与える方法
n_rep = len(violation_result["sorted_eigenvalues_list"][0])
n = num_data[0]
title = f"N={n}, Nrep={n_rep}, Number of Unphysical estimates={len(value_list)}"
fig.update_layout(title=title)
fig.update_xaxes( title=f"Sum of unphysical eigenvalues (<0)")
fig.show()

value_list = violation_result["sum_of_eigenvalues"]["greater_than_one"]
fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.0001, annotation_vlines=[1])

n_rep = len(violation_result["sorted_eigenvalues_list"][0])
n = num_data[0]
title = f"N={n}, Nrep={n_rep}, Number of Unphysical estimates={len(value_list)}"
fig.update_layout(title=title)  # TODO
fig.update_xaxes( title=f"Sum of unphysical eigenvalues (>1)")
fig.show()